In [ ]:

import os
from matplotlib import pyplot as plt
import torch
import cupy as cp


In [ ]:
os.chdir("/mnt/c/Users/smatysik/projects/nvImageCodecsWSL2/build/example/example_decode")
import nvimgcodecs
import nvcv



In [ ]:
nv_img_cat = nvimgcodecs.imread("cat.j2k")

In [ ]:
print(nv_img_cat.__cuda_array_interface__)
nvcvInputTensor = nvcv.as_tensor(nv_img_cat, "CHW")
print(nvcvInputTensor.cuda().__cuda_array_interface__)

# The Reformat operator can be used to convert CHW format to NHWC
# for the rest of the preprocessing operations
nvcvInterleavedTensor = nvcvInputTensor.reformat("NHWC")
print(nvcvInterleavedTensor.cuda().__cuda_array_interface__)

nvcvResizeTensor = nvcvInterleavedTensor#.resize((1, 320, 240, 3), nvcv.Interp.CUBIC)
print(nvcvResizeTensor.cuda().__cuda_array_interface__)
nvcvResizeTensorCHW = nvcvResizeTensor.reformat("CHW")
print(nvcvResizeTensorCHW.cuda().__cuda_array_interface__)
#tr_img = torch.as_tensor(nvcvInputTensor.cuda(), device="cuda")
ca = nvcvResizeTensorCHW.cuda().__cuda_array_interface__
#ca = {'shape':tuple(ca['shape']), 'strides':tuple(ca['strides']), 'typestr':ca['typestr'],'data':ca['data'], 'version':2}
ca = {'shape':tuple(ca['shape']), 'strides':tuple(ca['strides']), 'typestr':'=B','data':ca['data'], 'version':2}

cp_img = cp.asarray(ca)
# cp_img = cp.asarray(nvcvResizeTensorHWC.cuda())  -> TypeError: Expected tuple, got list
#tr_img = torch.as_tensor(nvcvResizeTensorCHW.cuda(), device="cuda")
#print(tr_img.cuda().__cuda_array_interface__)

#cp_img = cp.asarray(tr_img.cuda())
# CHW(c-style)  =Planar -> HWC (c-style) =Interleave
print(cp_img.__cuda_array_interface__)
cp_img = cp.moveaxis(cp_img, (2, 1, 0), (1, 0, 2))
cp_img = cp.asfortranarray(cp_img) # In Fortran the most rapidly changing index comes first
cp_img = cp.ascontiguousarray(cp_img)
print(cp_img.__cuda_array_interface__)

image = cp.asnumpy(cp_img)
plt.imshow(image)
